# Modeling bicycle counts: A Bayesian approach

In a previous post (add link), we have joined the Rostock bicycle counter data with weather data and public holidays. We have built a  basic linear model that explained our data rather reasonably -- apart from some outliers, which could be associated e.g. with the Hansa Sail Regatta, or public holidays like Christmas). This model was also able to predict cyclist counts on unseen data (from 2016, in our case).

This linear model was not only able to describe cyclist counts over the course of the year, but also allowed deducing e.g. how temperature or sunshine time affected cyclist counts: the trained model revealed an increase in daily cyclists counts by XX for each degree of temperature increase.

A conventional linear model, such as we used in this previous post, does not allow for a straightforward estimation of parameter uncertainty. In fact, it yields a point estimate, such as an increase of 80 per degree temperature increase. Often, one would like to know not only a point estimate, but also an estimate of the precision of this estimate - or, even better, the whole distribution of that parameter that likely produced the observed data.

Recently, a whole branch of statistics allowing precisely this estimation of underlying parameter distributions  has emerged and gained considerable traction. This branch of statistics is commonly referred to as 'Bayesian statistics'. This is not the place to provide an introduction into Bayesian reasoning, instead, I point the interested reader e.g. to a series of blog posts by Jake van der Plas (http://jakevdp.github.io/blog/2014/03/11/frequentism-and-bayesianism-a-practical-intro/) , or to an introductory textbook by John Kruschke (https://sites.google.com/site/doingbayesiandataanalysis/).

In the following, we use a Bayesian approach to estimate the distributions (or the uncertainty) of the parameters in a linear model explaining cyclist counts in Rostock downtown. We use the same data and the same linear model as in the previous post, and we switch from R to python, in order to use the excellent pymc3 library (https://pymc-devs.github.io/pymc3/notebooks/getting_started.html).

This post is written in a jupyter notebook, using anaconda python 2.7.12.

## Libraries and data

In [1]:
import pandas as pd
import numpy as np

# import pymc3 as pm

import seaborn as sns

%matplotlib notebook

In [2]:
daily = pd.read_csv('../data/processed/train.daily.csv', index_col='date',
                   parse_dates = True)

# we introduce the long-awaited 'work day' column
daily['workday'] =(daily.index.dayofweek<5) & ~daily.holiday

# and we dump all columns which we do not need:
daily.drop(['weekday','precipitation', 'Feiertage', 'holiday'], axis = 1, inplace = True)

daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 712 entries, 2013-01-01 to 2014-12-31
Data columns (total 5 columns):
n               712 non-null int64
temperature     712 non-null float64
sunshinetime    649 non-null float64
rainy_day       712 non-null bool
workday         712 non-null bool
dtypes: bool(2), float64(2), int64(1)
memory usage: 23.6 KB


In [ ]:
from pymc3 import Model, sample

from pymc3.glm import glm

with Model() as model_glm:
    glm('n ~ temperature + sunshinetime', daily)
    trace = sample(5000)

Applied log-transform to sd and added transformed sd_log_ to model.
Assigned NUTS to Intercept
Assigned NUTS to temperature
Assigned NUTS to sunshinetime
Assigned NUTS to sd_log_
  2%|▋                                        | 77/5000 [11:10<43:38:58, 31.92s/it]

In [ ]:
# this is silly. we define our model properly:
from pymc3 import Model, sample, Poisson, Normal, HalfNormal,find_MAP
with Model() as model:
    temperature = Normal('temperature', mu = 0, sd =100) # uninformative # or informative?
    sunshinetime = Normal('sunshinetime', mu = 0, sd =100) # more sunshinetime equals more cyclists, right=
    sigma = HalfNormal('sigma', sd = 100)
    intercept= Poisson('intercept', mu = 10)
    expected_count = Poisson('cyclist count', 
                             intercept + temperature + sunshinetime)
    count_observed = Poisson('observed count', 
                             mu = expected_count,
                            observed = daily.n)
    start = find_MAP()

    # draw 2000 posterior samples
    trace = sample(2000, start=start)
Poisson()

Applied log-transform to sigma and added transformed sigma_log_ to model.
Assigned NUTS to temperature
Assigned NUTS to sunshinetime
Assigned NUTS to sigma_log_
Assigned Metropolis to intercept
Assigned Metropolis to cyclist count
  0%|                                                     | 0/2000 [00:00<?, ?it/s]